with -r, will that affect the normal lines of simulation tho, like when you pass the -r with the .rou.xml file, the SUMO is like "oh you passed a route file, and this file includes just a route and a car in that route, so that I will spawn just that car following thar route", isn't it? Or actually, idk if my map file contains a specific veh-route .xml file at all, but the overall is if -r affect the naturalness of simulation?

You are absolutely correct to worry about this. This is a classic SUMO configuration "gotcha."

**The Short Answer:**
Yes. If you run `sumo -c osm.sumocfg -r test_route.rou.xml`, SUMO will **replace** the background traffic defined in your `osm.sumocfg` with your new file. You will end up with an empty city and just your one car.

**The Explanation:**
Your `osm.sumocfg` file contains a line inside it that looks like this:
```xml
<route-files value="osm.passenger.trips.xml,osm.motorcycle.trips.xml,..."/>
```
When you provide the `-r` (or `--route-files`) flag in your Python script, you are telling SUMO: *"Ignore what is in the config file for routes, and use THIS file instead."*

### How to Fix It (To keep background traffic AND your test car)

You have two easy options.

#### Option 1: Use the `-a` (Additional) flag (Recommended for you)
In SUMO, you can load route files as "additional files." SUMO will load the main config (Map + Background Traffic) and *then* add your test car on top of it.

**In your Python `reset` method:**
```python
if self.test_mode:
    active_map = self.maps[0]
    # Use "-a" (Additional) instead of "-r". 
    # This ADDS your car to the existing simulation instead of replacing it.
    route_arg = ["-a", self.test_route] 
else:
    # ...
```

#### Option 2: Create a Dedicated Test Config (Best for Research)
If you want to be very professional, you create a new config file called `test.sumocfg` inside your folder.
In that file, you copy the contents of `osm.sumocfg` but **add** your test route to the list.

*Inside `test.sumocfg`:*
```xml
<configuration>
    <input>
        <net-file value="osm.net.xml.gz"/>
        <!-- Add your file to the end of this list -->
        <route-files value="osm.passenger.trips.xml,osm.motorcycle.trips.xml,test_route.rou.xml"/> 
        <additional-files value="osm.poly.xml.gz,..."/>
    </input>
</configuration>
```
Then in Python, you just load `active_map = "maps/TestMap/test.sumocfg"` and pass no extra arguments.

### Summary
Since you want to stick to the Python script focus right now, **go with Option 1**.

Change the line in your `reset` method back to `-a`:
```python
route_arg = ["-a", self.test_route]
```
This tells SUMO: "Load the map and traffic as usual, and **additionally** spawn my test vehicle."